In [2]:
import numpy as np
import pandas as pd
import talib
from talib import abstract
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# 整理 maingroup_raw.csv 

In [ ]:
maingroup = pd.read_csv('maingroup_raw.csv')  
a = maingroup[['tmp_index_ts_value_20200430_henry.index_nm', 'tmp_index_ts_value_20200430_henry.tx_dt', 'tmp_index_ts_value_20200430_henry.close']]
a = a.sort_values(by = 'tmp_index_ts_value_20200430_henry.tx_dt').reset_index(drop = True)
a.columns = ['name', 'tx_dt', 'close']
a
a.to_csv('maingroup.csv', index = False)

# 整理 eco_data_raw.csv

In [ ]:
# 經濟指標
eco_df = pd.read_csv('eco_data.csv')  
index_name = np.unique(eco_df[NAME])
eco_idx_drop_lst = []
for name in index_name:
    num = np.sum(np.array(eco_df[eco_df[NAME] == name].isna()))
    if num:
        eco_idx_drop_lst.append(name)

# 刪除過多 nan 的經濟指標
eco_df.drop(index = np.where(eco_df[NAME].isin(eco_idx_drop_lst))[0], inplace = True)
eco_df.reset_index(drop = True, inplace = True)
eco_df.to_csv('eco_data.csv', index = False)

# Feature engineering

In [3]:
def feature_engineering(market_name, maingroup, eco_df, NAME = 'name', TXDT = 'tx_dt', OPEN = 'open', HIGH = 'high', LOW = 'low', CLOSE = 'close'):
    # ---------------Parameter---------------
    # | market_name: 目標分析市場名字          |
    # | maingroup: 含所有市場資料的 dataframe  |
    # | eco_df: 含所有經濟指標的 dataframe     |
    # | ...                                 |
    # ---------------------------------------
    
    # Ta_lib
    talib_func_list = talib.get_functions()

    # 生成目標市場之經濟指標

    target_df = maingroup[maingroup[NAME] == market_name][[TXDT, CLOSE]]
    target_df.sort_values(by = TXDT, inplace = True)
    target_df.reset_index(drop = True, inplace = True)
    target_df

    for ta_func in talib_func_list:
        try:
            new_feature = eval('abstract.' + ta_func + '(target_df, timeperiod=22)')
            target_df[ta_func + '_' + market_name] = new_feature
        except:
            continue

    # ---------------------------------------------------

    # 生成經濟指標與其技術指標
    # 產生第一個
    eco_name_lst = np.unique(eco_df[NAME])
    name = eco_name_lst[0]
    total_eco = eco_df[eco_df[NAME] == name][[TXDT, OPEN, HIGH, LOW, CLOSE]]
    for ta_func in talib_func_list:
        try:
            new_feature = eval('abstract.' + ta_func + '(total_eco, timeperiod=22)')
            total_eco[ta_func + '_' + name] = new_feature
        except:
            continue

    col_name = []
    for i in range(1, total_eco.columns.shape[0]):
        col_name.append(total_eco.columns[i] + '_' + name)
    total_eco.columns = [TXDT] + col_name

    # Start...
    for name in eco_name_lst[1: ]:
        target_eco = eco_df[eco_df[NAME] == name][[TXDT, OPEN, HIGH, LOW, CLOSE]]

        for ta_func in talib_func_list:
            try:
                new_feature = eval('abstract.' + ta_func + '(target_eco, timeperiod=22)')
                target_eco[ta_func + '_' + name] = new_feature
            except:
                continue

        col_name = []
        for i in range(1, target_eco.columns.shape[0]):
            col_name.append(target_eco.columns[i] + '_' + name)
        target_eco.columns = [TXDT] + col_name

        total_eco = pd.merge(total_eco, target_eco, on = TXDT)

    # Concat 目標市場 與 經濟指標
    Total_df = pd.merge(target_df, total_eco, on = TXDT)
    
    return Total_df

In [4]:
maingroup = pd.read_csv('maingroup.csv')
eco_df = pd.read_csv('eco_data.csv')

In [7]:
total_df = feature_engineering(market_name = 'MSCI China NET TOTAL RETURN LOCAL',
                               maingroup = maingroup, eco_df = eco_df)
print(total_df.shape)

(3821, 2069)


In [15]:
np.isnan(np.array(total_df.drop(columns = ['tx_dt']))).sum()

112939